In [41]:
import pandas as pd
import numpy as np
import os
import sys

### Read the data

In [42]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [43]:
def display_int_grid(grid):
    # grid is an nd array with 1s and 0s, display it with '.' for 0 and '#' for 1
    for row in grid:
        for col in row:
            if col == 0:
                print('.', end='')
            else:
                print('#', end='')
        print('')


def char_grid_to_int_grid(char_grid):
    # convert to a numpy array where '.' is 0 and '#' is 1
    int_grid = np.array([list(x.replace('.', '0').replace('#', '1')) for x in char_grid])
    # convert to int
    int_grid = int_grid.astype(int)

    # the first 1 stays 1, the second 1 becomes 2, etc.
    # get the unique values in the grid
    unique_vals = np.unique(int_grid)
    # for each unique value
    for unique_val in unique_vals:
        # if it's not 0
        if unique_val != 0:
            # replace it with its index
            int_grid[int_grid == unique_val] = np.where(unique_vals == unique_val)[0][0]

    return int_grid

def expand_grid(grid):
    # add columns
    # get the sum of each column
    col_sums = np.sum(grid, axis=0)
    # get the indices of the columns that sum to zero
    zero_col_indices = np.where(col_sums == 0)[0]

    # add columns
    # if there are columns that sum to zero
    while len(zero_col_indices) > 0:
        # insert a column of zeros at that index
        # get the lowest index
        zero_col_index = min(zero_col_indices)
        grid = np.insert(grid, zero_col_index, 0, axis=1)
        # drop the index from the list
        zero_col_indices = np.delete(zero_col_indices, np.where(zero_col_indices == zero_col_index)[0][0])

        if len(zero_col_indices) > 0:
            # add 1 to all the indices that are higher than the one we just added
            zero_col_indices = zero_col_indices + 1
        
    # add rows
    # get the sum of each row
    row_sums = np.sum(grid, axis=1)
    # get the indices of the rows that sum to zero
    zero_row_indices = np.where(row_sums == 0)[0]

    # add rows
    # if there are rows that sum to zero
    while len(zero_row_indices) > 0:
        # for each row that sums to zero
        zero_row_index = min(zero_row_indices)
        # insert a row of zeros at that index
        grid = np.insert(grid, zero_row_index, 0, axis=0)

        # drop the index from the list
        zero_row_indices = np.delete(zero_row_indices, np.where(zero_row_indices == zero_row_index)[0][0])

        if len(zero_row_indices) > 0:
            # add 1 to all the indices that are higher than the one we just added
            zero_row_indices = zero_row_indices + 1
    
    return grid


def manhattan_distance(point1, point2):
    # point1 and point2 are tuples of the form (y, x)
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])


def get_sum_of_distances(grid):
    working_grid = grid.copy()

    # vind the first 1 in the grid
    all_ones = np.where(working_grid == 1)
    all_ones = list(zip(all_ones[0], all_ones[1]))
    
    distances = list()
    for i, one in enumerate(all_ones):
        # only look at the ones after this one
        for j, other_one in enumerate(all_ones[i+1:]):
            distance = manhattan_distance(one, other_one)
            distances.append(distance)
    return sum(distances)

In [44]:
working_file = test_file_content

grid = char_grid_to_int_grid(working_file)
grid = expand_grid(grid)
sum_of_distances = get_sum_of_distances(grid)

print(sum_of_distances)

374


### Part 2

In [56]:
def get_sum_of_distances_empty(grid, n_times_empty=1):
    working_grid = grid.copy()

    empty_columns = np.where(np.sum(working_grid, axis=0) == 0)[0]
    empty_rows = np.where(np.sum(working_grid, axis=1) == 0)[0]

    # vind the first 1 in the grid
    all_ones = np.where(working_grid == 1)
    all_ones = list(zip(all_ones[0], all_ones[1]))
    
    distances = list()
    for i, one in enumerate(all_ones):
        # only look at the ones after this one
        for j, other_one in enumerate(all_ones[i+1:]):
            distance = manhattan_distance(one, other_one)
            # count the number of empty columns between the two points
            empty_columns_between = len([x for x in empty_columns if (x > one[1] and x < other_one[1]) or (x > other_one[1] and x < one[1])])
            # count the number of empty rows between the two points
            empty_rows_between = len([x for x in empty_rows if (x > one[0] and x < other_one[0]) or (x > other_one[0] and x < one[0])])
            distance = distance + (n_times_empty-1) * (empty_columns_between + empty_rows_between)
            distances.append(distance)
    return sum(distances)


working_file = file_content
grid = char_grid_to_int_grid(working_file)

sum_of_distances = get_sum_of_distances_empty(grid, n_times_empty=1000000)

print(sum_of_distances)

544723432977
